In [18]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('June11Assignment3').getOrCreate()

spark


In [19]:
data = [
("Ananya", "HR", 52000),
("Rahul", "Engineering", 65000),
("Priya", "Engineering", 60000),
("Zoya", "Marketing", 48000),
("Karan", "HR", 53000),
("Naveen", "Engineering", 70000),
("Fatima", "Marketing", 45000)
]
columns = ["Name", "Department", "Salary"]
df_emp = spark.createDataFrame(data, columns)

performance = [
("Ananya", 2023, 4.5),
("Rahul", 2023, 4.9),
("Priya", 2023, 4.3),
("Zoya", 2023, 3.8),
("Karan", 2023, 4.1),
("Naveen", 2023, 4.7),
("Fatima", 2023, 3.9)
]
columns_perf = ["Name", "Year", "Rating"]
df_perf = spark.createDataFrame(performance, columns_perf)

project_data = [
("Ananya", "HR Portal", 120),
("Rahul", "Data Platform", 200),
("Priya", "Data Platform", 180),
("Zoya", "Campaign Tracker", 100),
("Karan", "HR Portal", 130),
("Naveen", "ML Pipeline", 220),
("Fatima", "Campaign Tracker", 90)
]
columns_proj = ["Name", "Project", "HoursWorked"]
df_proj = spark.createDataFrame(project_data, columns_proj)

In [20]:
#Joins and Advanced Aggregations
#1. Join employee_data ,  performance_data , and project_data.
df_joined = df_emp.join(df_perf, on="Name", how="outer")
df_final = df_joined.join(df_proj, on="Name", how="outer")
df_final.show()
#2. Compute total hours worked per department.
from pyspark.sql.functions import sum
df_hours_worked = df_final.groupBy("Department").agg(sum("HoursWorked").alias("TotalHoursWorked"))
df_hours_worked.show()
#3. Compute average rating per project.
from pyspark.sql.functions import avg
df_avg_rating = df_final.groupBy("Project").agg(avg("Rating").alias("AverageRating"))
df_avg_rating.show()

+------+-----------+------+----+------+----------------+-----------+
|  Name| Department|Salary|Year|Rating|         Project|HoursWorked|
+------+-----------+------+----+------+----------------+-----------+
|Ananya|         HR| 52000|2023|   4.5|       HR Portal|        120|
|Fatima|  Marketing| 45000|2023|   3.9|Campaign Tracker|         90|
| Karan|         HR| 53000|2023|   4.1|       HR Portal|        130|
|Naveen|Engineering| 70000|2023|   4.7|     ML Pipeline|        220|
| Priya|Engineering| 60000|2023|   4.3|   Data Platform|        180|
| Rahul|Engineering| 65000|2023|   4.9|   Data Platform|        200|
|  Zoya|  Marketing| 48000|2023|   3.8|Campaign Tracker|        100|
+------+-----------+------+----+------+----------------+-----------+

+-----------+----------------+
| Department|TotalHoursWorked|
+-----------+----------------+
|Engineering|             600|
|         HR|             250|
|  Marketing|             190|
+-----------+----------------+

+----------------+----

In [21]:
#Handling Missing Data (introduce some manually)
#4. Add a row to performance_data with a None rating.
from pyspark.sql.functions import lit,when
df_perf_with_none = df_perf.withColumn("Rating", lit(None))
df_perf_with_none.show()
#5. Filter rows with null values.
df_null_filtered = df_perf_with_none.filter(df_perf_with_none["Rating"].isNull())
df_null_filtered.show()
#6. Replace null ratings with the department average.
from pyspark.sql.window import Window
df_replaced_null= df_final.withColumn("Rating", when(df_final["Rating"].isNull(), avg(df_final["Rating"]).over(Window.partitionBy("Department"))).otherwise(df_final["Rating"]))
df_replaced_null.show()

+------+----+------+
|  Name|Year|Rating|
+------+----+------+
|Ananya|2023|  NULL|
| Rahul|2023|  NULL|
| Priya|2023|  NULL|
|  Zoya|2023|  NULL|
| Karan|2023|  NULL|
|Naveen|2023|  NULL|
|Fatima|2023|  NULL|
+------+----+------+

+------+----+------+
|  Name|Year|Rating|
+------+----+------+
|Ananya|2023|  NULL|
| Rahul|2023|  NULL|
| Priya|2023|  NULL|
|  Zoya|2023|  NULL|
| Karan|2023|  NULL|
|Naveen|2023|  NULL|
|Fatima|2023|  NULL|
+------+----+------+

+------+-----------+------+----+------+----------------+-----------+
|  Name| Department|Salary|Year|Rating|         Project|HoursWorked|
+------+-----------+------+----+------+----------------+-----------+
|Naveen|Engineering| 70000|2023|   4.7|     ML Pipeline|        220|
| Priya|Engineering| 60000|2023|   4.3|   Data Platform|        180|
| Rahul|Engineering| 65000|2023|   4.9|   Data Platform|        200|
|Ananya|         HR| 52000|2023|   4.5|       HR Portal|        120|
| Karan|         HR| 53000|2023|   4.1|       HR Port

In [22]:
#Built-In Functions and UDF
#7. Create a column PerformanceCategory :
# Excellent (>=4.7),
# Good (4.0–4.69),
# Average (<4.0)
from pyspark.sql.functions import when
df_performance_category = df_final.withColumn("PerformanceCategory", when(df_final["Rating"] >= 4.7, "Excellent").otherwise(when(df_final["Rating"] >= 4.0, "Good").otherwise("Average")))
df_performance_category.show()
#8. Create a UDF to assign bonus:
#If project hours > 200 → 10,000
#Else → 5,000
def assign_bonus(hours_worked):
    if hours_worked > 200:
        return 10000
    else:
        return 5000
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
udf_assign_bonus = udf(assign_bonus, IntegerType())
df_with_bonus = df_final.withColumn("Bonus", udf_assign_bonus(df_final["HoursWorked"]))
df_with_bonus.show()


+------+-----------+------+----+------+----------------+-----------+-------------------+
|  Name| Department|Salary|Year|Rating|         Project|HoursWorked|PerformanceCategory|
+------+-----------+------+----+------+----------------+-----------+-------------------+
|Ananya|         HR| 52000|2023|   4.5|       HR Portal|        120|               Good|
|Fatima|  Marketing| 45000|2023|   3.9|Campaign Tracker|         90|            Average|
| Karan|         HR| 53000|2023|   4.1|       HR Portal|        130|               Good|
|Naveen|Engineering| 70000|2023|   4.7|     ML Pipeline|        220|          Excellent|
| Priya|Engineering| 60000|2023|   4.3|   Data Platform|        180|               Good|
| Rahul|Engineering| 65000|2023|   4.9|   Data Platform|        200|          Excellent|
|  Zoya|  Marketing| 48000|2023|   3.8|Campaign Tracker|        100|            Average|
+------+-----------+------+----+------+----------------+-----------+-------------------+

+------+-----------+

In [23]:
#Date and Time Functions
#9. Add a column JoinDate with 2021-06-01 for all, then add MonthsWorked as difference from today.
from pyspark.sql.functions import lit, datediff, current_date
df_with_join_date = df_final.withColumn("JoinDate", lit("2021-06-01"))
df_with_months_worked = df_with_join_date.withColumn("MonthsWorked", datediff(current_date(), "JoinDate") / 30)
df_with_months_worked.show()
# 10. Calculate how many employees joined before 2022.
from pyspark.sql.functions import col
df_before_2022 = df_final.filter(col("Year") < 2022)
count_before_2022 = df_before_2022.count()
print("Number of employees joined before 2022:", count_before_2022)

+------+-----------+------+----+------+----------------+-----------+----------+-----------------+
|  Name| Department|Salary|Year|Rating|         Project|HoursWorked|  JoinDate|     MonthsWorked|
+------+-----------+------+----+------+----------------+-----------+----------+-----------------+
|Ananya|         HR| 52000|2023|   4.5|       HR Portal|        120|2021-06-01|49.03333333333333|
|Fatima|  Marketing| 45000|2023|   3.9|Campaign Tracker|         90|2021-06-01|49.03333333333333|
| Karan|         HR| 53000|2023|   4.1|       HR Portal|        130|2021-06-01|49.03333333333333|
|Naveen|Engineering| 70000|2023|   4.7|     ML Pipeline|        220|2021-06-01|49.03333333333333|
| Priya|Engineering| 60000|2023|   4.3|   Data Platform|        180|2021-06-01|49.03333333333333|
| Rahul|Engineering| 65000|2023|   4.9|   Data Platform|        200|2021-06-01|49.03333333333333|
|  Zoya|  Marketing| 48000|2023|   3.8|Campaign Tracker|        100|2021-06-01|49.03333333333333|
+------+-----------+

In [26]:
#Unions
#11. Create another small team DataFrame and union() it with employee_data.
#extra_employees = [
#("Meena", "HR", 48000),
#("Raj", "Marketing", 51000)
#]
extra_employees = [
("Meena", "HR", 48000),
("Raj", "Marketing", 51000)
]
columns_extra = ["Name", "Department", "Salary"]
df_extra_employees = spark.createDataFrame(extra_employees, columns_extra)
df_unioned = df_emp.union(df_extra_employees)
df_unioned.show()


+------+-----------+------+
|  Name| Department|Salary|
+------+-----------+------+
|Ananya|         HR| 52000|
| Rahul|Engineering| 65000|
| Priya|Engineering| 60000|
|  Zoya|  Marketing| 48000|
| Karan|         HR| 53000|
|Naveen|Engineering| 70000|
|Fatima|  Marketing| 45000|
| Meena|         HR| 48000|
|   Raj|  Marketing| 51000|
+------+-----------+------+



In [27]:
#Saving Results
#12.Save the final merged dataset (all 3 joins) as a partitioned Parquet file based on Department .
df_final.write.partitionBy("Department").parquet("df_final_partitioned.parquet")


In [28]:
spark.stop()